<a href="https://colab.research.google.com/github/zulfanaliy/TM/blob/main/cleansing_data_tokped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install emoji


Install Libraries

In [21]:
pip install contractions

In [22]:
import pandas as pd
import emoji
import contractions
import re
import nltk

In [23]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

Menghilangkan karakter yang tidak dibutuhkan

In [28]:
def cleansing(Ulasan) :
    # Replace RT tag
    t1 = re.sub('RT\s', '', Ulasan)
    # Replace @_username
    t2 = re.sub('\B@\w+', '', t1)
    # Replace emojis with text
    t3 = emoji.demojize(t2)
    # Replace URL (http:// or https://)
    t4 = re.sub('(http|https):\/\/\S+', '', t3)
    # Replace #_something_
    t5 = re.sub('#+', '', t4)
    # Lower case each Letter
    t6 = t5.lower()
    # Replace word repetition with a single Bccurance ('oooooo' becomse 'oo')
    t7 = re.sub(r'(.)\1+', r'\1\1', t6)
    # Replace punctuation repetition with a single occurance ('!!!!!!!!!' becomes '!')
    t8 = re.sub(r'[\?\.\!]+(?=[\?.\!])', '', t7)
    # Alphabet only, exclude special characters
    t9 = re.sub('[^a-zA-Z0-9]', ' ', t8)
    # Replace contractions with their extended forms
    t10 = contractions.fix(t9)
    return t10

Mengambil data raw dari crawling twitter

In [29]:
df = pd.read_csv('Ulasan di tokped_raw.csv', usecols=['Nama_Produk', 'Akun', 'Ulasan', 'rate'])
df.head()


,Nama_Produk,Akun,Ulasan,rate
0,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Stephanus,keduakali beli tapi sayang yang kedua lambat d...,3
1,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Yati,bagus pengiriman cepat,5
2,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Husin,barang sesuai pesanan,4
3,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Febby,agak panas pas dipakai,5
4,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Rahmat,barang sesuai permintaan dan bagus namun komun...,3


In [32]:
df ['Ulasan'] = df['Ulasan'].astype(str)

Menghilangkan simbol https, dan tag @

In [33]:
for i, r in df.iterrows():
    y = cleansing(r['Ulasan'])
    df.loc[i, 'Ulasan'] = y

df['Ulasan'] = df['Ulasan'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
df['Ulasan'] = df['Ulasan'].replace(r'@\S+', '', regex=True)

df.head()

,Nama_Produk,Akun,Ulasan,rate
0,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Stephanus,keduakali beli tapi sayang yang kedua lambat d...,3
1,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Yati,bagus pengiriman cepat,5
2,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Husin,barang sesuai pesanan,4
3,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Febby,agak panas pas dipakai,5
4,Masker KF94 4 Ply Isi 10 Pcs Masker Medis Kese...,Rahmat,barang sesuai permintaan dan bagus namun komun...,3


Export hasil cleansing ke csv

In [34]:
df.to_csv('data_cleansed_tokped.csv', index=False)